<a href="https://colab.research.google.com/github/rana14ce/Dataset-of-the-Multinational-Pile/blob/v1/GitHub_XGBoost_with_5_fold_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# CTRL+Shift+Space bar = docstring
# CTRL+Space bar = autocomplete

# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

In [14]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [15]:
url = 'https://raw.githubusercontent.com/rana14ce/Dataset-of-the-Multinational-Pile/main/Dataset%20of%20the%20Multinational%20Pile.csv'
data = pd.read_csv(url)

In [16]:
data.head()

,Phi_s,Phi_t,L_m,A_m2,Sv _KN/m2,Qm_KN
0,33.0,38.0,255,24.5,0.1310,2615
1,34.0,37.5,206,19.8,0.2230,3675
2,33.0,38.0,223,21.5,0.1310,2164
3,33.0,37.5,210,20.2,0.1468,3042
4,33.0,37.0,206,19.9,0.1821,2856


In [17]:
X = data.drop('Qm_KN', axis=1)
y = data['Qm_KN']

In [18]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
# Initialize the XGBoost Regressor
xgb_model = XGBRegressor()

In [21]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [3, 5, 7],
    'gamma': [0, 0.25, 0.5],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Perform GridSearchCV
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 81 candidates, totalling 405 fits


In [22]:
# Perform 5-Fold Cross-Validation with the best model
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores)
mean_cv_rmse = np.mean(cv_rmse)

In [23]:
# Train the best model on the entire training set
best_model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [24]:
# Predict on the training set
y_train_pred = best_model.predict(X_train_scaled)

# Predict on the test set
y_test_pred = best_model.predict(X_test_scaled)

In [25]:
# Evaluate the model on the training set
train_r2 = r2_score(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

# Evaluate the model on the test set
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("Best Parameters: ", best_params)
print("5-Fold CV RMSE: ", cv_rmse)
print("Mean CV RMSE: ", mean_cv_rmse)
print("Training R^2: ", train_r2)
print("Training RMSE: ", train_rmse)
print("Test R^2: ", test_r2)
print("Test RMSE: ", test_rmse)


Best Parameters:  {'gamma': 0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200}
5-Fold CV RMSE:  [ 912.09585042  529.56187968  989.45781388  839.76125198 1006.80746903]
Mean CV RMSE:  855.5368529974852
Training R^2:  0.9097672134721765
Training RMSE:  343.072655492975
Test R^2:  0.19905420894991854
Test RMSE:  1232.351645936157
